# Create and connect to local database

In [1]:
import pandas as pd
import sqlite3
from sqlite3 import Error
from data_storage import connection

OperationalError: unable to open database file

In [ ]:
df = pd.read_sql_query('''SELECT * FROM hours''', connection)

In [2]:
pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip")

ValueError: Multiple files found in compressed zip file ['Readme.txt', 'day.csv', 'hour.csv']

This will connect to the database and if it does not exist, it will create one.

In [25]:
def create_connection(db_name):
    """ create a database connection to the SQLite database
        specified by db_name
    :param db_name: database name
    :return: Connection object or None
    """
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    return c, conn

In [26]:
c, conn = create_connection("BikeRental.db")


Everything within a database is stored as a table. So lets create a function which checks whether a table is already exsiting within a database.

In [27]:
def table_exists(table_name): 
    """
    """
    c.execute('''SELECT count(name) FROM sqlite_master WHERE TYPE = 'table' AND name = '{}' '''.format(table_name)) 
    if c.fetchone()[0] == 1: 
        return True 
    return False

def check_and_create(conn, table_name, sql_table_creating_string):
    """
    """
    if table_exists(table_name):
        print("Table already exists!")
        return None
    else:
        conn.execute(sql_table_creating_string)
        print("Table is created!")
        return None

In [28]:
sql_table_creating_string = ''' 
        CREATE TABLE hourlys( 
            instant INTEGER, 
            dteday DATE, 
            season INTEGER, 
            yr INTEGER, 
            mnth INTEGER,
            hr INTEGER,
            holiday INTEGER,
            weekday INTEGER,
            workingday INTEGER,
            weathersit INTEGER,
            temp REAL,
            atemp REAL,
            hum REAL,
            windspeed INTEGER,
            casual INTEGER,
            registered INTEGER,
            cnt INTEGER 
        ) 
    '''

In [29]:
check_and_create(conn, "hourlys", sql_table_creating_string)

Table is created!


True

In [30]:
def insert_values_to_table(conn, table_name, csv_file):
    """
    Open a csv file with pandas, store its content in a pandas data frame, change the data frame headers to the table
    column names and insert the data to the table
    :param table_name: table name in the database to insert the data into
    :param csv_file: path of the csv file to process
    :return: None
    """
    c = conn.cursor()

    df = pd.read_csv(csv_file)

    df.to_sql(name=table_name, con=conn, if_exists='append', index=False)

    print('SQL insert process finished')
    return None

In [31]:
insert_values_to_table(conn, "hourlys", "../data/raw/hour.csv")

SQL insert process finished


In [32]:
def get_data(c, table_name): 
    c.execute('''SELECT * FROM {}'''.format(table_name)) 
    data = [] 
    for row in c.fetchall(): 
        data.append(row) 
    return data

In [35]:
pd.read_sql_query('''SELECT * FROM hourlys''', conn)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119
17375,17376,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89
17376,17377,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90
17377,17378,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48,61
